In [ ]:
few shots prompting: provide  high quality example of the output that is desired

In [2]:
import os
import litellm
import json
from dotenv import load_dotenv
from textwrap import dedent
from IPython.display import display, Markdown

load_dotenv()

print("imported")

AI_MODEL="openai/gpt-4o-mini" # OR: openai/gpt-4o-mini /anthropic/claude-haiku-4-5-20251001/ change between them to observe the effects!
MAX_TOKENS_DEFAULT = 500

def get_completion(prompt_or_messages, model=AI_MODEL, max_tokens=MAX_TOKENS_DEFAULT, **kwargs):
    # If user passes a plain string, wrap it in a messages list
    if isinstance(prompt_or_messages, str):
        messages = [{"role": "user", "content": prompt_or_messages}]
    else:
        messages = prompt_or_messages

    # Guard: avoid sending empty messages
    if not isinstance(messages, list) or len(messages) == 0:
        raise ValueError(f"'messages' must be a non-empty list. Got: {messages}")

    resp = litellm.completion(
        model=model,
        messages=messages,
        max_tokens=max_tokens,
        **kwargs
    )
    return resp.choices[0].message.content

print("All set!")

imported
All set!


In [3]:
user_query = "Can you find my ticket #T-009?"

In [23]:
#Zero-shot: no example provided that describe the output. Only instructions, Context, Rules, Setting. 

zero_shot_messages = [
    {
    "role": "system",
    "content": dedent("""
    You are a command processing system and your job is to convert a user query into a JSON object.
    """)
    },
    {
    "role": "user",
    "content": user_query
    }
]

response_zero_shot = get_completion(zero_shot_messages, response_format={"type": "json_object"})

In [24]:
print(response_zero_shot)

{
  "query": "find ticket",
  "ticket_id": "T-009"
}


In [30]:
# Few-shot prompt

few_shot_messages = [
    {
         "role": "system",
    "content": dedent("""
    You are a command processing system and your job is to convert a user query into a JSON object.
    """)
    },
    #Example 1
    {
    "role": "user",
    "content": "Show me the latest news about AI(Open AI, Anhropic, Gemini"
    },
    {
     "role": "assistant",
     "content": json.dumps({ "action": "fetch_news", "params":{ "entity": "Anthropic" }})
    },
     #Example 2
    {
    "role": "user",
    "content": "What's the weather in Europe?"
    },
    {
     "role": "assistant",
    "content": json.dumps({"action": "get_weather", "params": {"entity": "Europe" }})
    },
    # Our new query
    {                      
     "role": "user",
    "content": "What's the weather in Europe?"                     
    }
]

response_few_shots = get_completion(
    few_shot_messages, response_format ={"type": "json_object"}
)


In [31]:
print(response_few_shots)

{"action": "get_weather", "params": {"region": "Europe"}}


In [32]:
# Few-shot prompt by providing bussines rules instead of code:

few_shot_messages_bussines = [
    {
         "role": "system",
    "content": dedent("""
    You are a command processing system and your job is to convert a user query into a JSON object.

    Follow the schema:
    {
        "actions" : <function to call"
        "params" : {
            "entity": <entity to research>
        }
    }
    """)
    },
    {                      
     "role": "user",
    "content": "What's the weather in Europe?"                     
    }
]

response_few_shots_bussines = get_completion(
    few_shot_messages_bussines, response_format ={"type": "json_object"}
)


In [33]:
print(response_few_shots_bussines)

{
    "actions": "getWeather",
    "params": {
        "entity": "Europe"
    }
}
